In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
from sklearn.metrics import accuracy_score

%matplotlib inline

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [26]:
### PREPROCESSING AND TAGGING

In [27]:
os.chdir("C://Users//Ruben//Documents//GitHub//dhl-ads//test_ads")
data = pd.read_csv('test-batch2.csv', sep = '\t')

In [28]:
def clean_and_split_str(txt):
    #strip_special_chars = re.compile("[^A-Za-z0-9#]+")
    translator = str.maketrans('', '', string.punctuation)
    txt = txt.translate(translator)
    txt = re.sub('\s+', ' ', txt).strip()
    txt = txt.lower()
    return txt

In [29]:
data['clean'] = [clean_and_split_str(art) for art in data.ocr]
data = data[['id', 'clean']]

In [30]:
ad_words = ['huishoudster', 'beschaafd', 'gezocht', 'goede getuigschriften', 'goede getuigen voorzien', 'geplaatst worden', 'goed kunnende']

In [60]:
### Classify Manually
data['class'] = ""

for i in range(0,len(data) - 1):
    if any(word in data.clean[i] for word in ad_words):
        data['class'][i] = 1
    else:
        data['class'][i] = 0
        

In [61]:
## Create Sets
train_x, valid_x, train_y, valid_y = train_test_split(data['clean'], data['class'])

In [82]:
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(data['clean'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x) 
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x) 

In [67]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    #predictions = classifier.predict(feature_vector_valid)
    predictions = classifier.predict(feature_vector_valid)
    return predictions
    
    #return metrics.accuracy_score(predictions, valid_y)

In [81]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)

NameError: name 'xtrain_tfidf_ngram' is not defined